# <span style="color:black;">GDAL in Python</span>
## <span style="color:black;">Overview</span>

By Vitor Martins, and revised by _____

We learned that you can use GDAL programs, and now we  in Python to work with geospatial data, and this is typically done using the gdal Python module.

```bash
conda create -n geoml python=3.7
conda activate geoml
pip install wheel
pip install pipwin
pipwin install numpy
pipwin install pandas
pipwin install shapely
pipwin install gdal

ou

conda install gdal

pipwin install fiona
pipwin install pyproj
pipwin install six
pipwin install rtree
pip install geopandas
pip install tensorflow-gpu
pip install opencv-python
pip install matplotlib
pipwin pip
conda install -c conda-forge scikit-learn
conda install -c conda-forge rasterio  ## rasterio needs conda-forge

```

## <span style="color:black;">Packages for GeoSpatial</span>

The GDAL installation is part of many software, including QGIS. I suggest the OSGeo4W for Windows. Go to the OSGeo4W website: https://trac.osgeo.org/osgeo4w/, select the 64-bit version (most computers), and run the installer. Once you finish it, you can open the command line and verify if your computer is "finding" the gdal programs (e.g., gdalinfo, gdal_translate, gdalwarp etc).


## <span style="color:black;">Exploring satellite image</span>





In [ ]:
from osgeo import gdal

dataset = gdal.Open("path/image.tif")
width = dataset.RasterXSize
height = dataset.RasterYSize
projection = dataset.GetProjection()
geotransform = dataset.GetGeoTransform()
band = dataset.GetRasterBand(1)  # Access the first band (bands are 1-based)
data = band.ReadAsArray()
dataset = None  # Close the dataset


In [ ]:
target_projection = "EPSG:4326"  # Define the target projection
reprojected_dataset = gdal.Warp("output.tif", dataset, dstSRS=target_projection)

In [ ]:
import matplotlib.pyplot as plt
from osgeo import gdal

# Open the single-band satellite image using GDAL
dataset = gdal.Open("path/to/single_band_image.tif")
band = dataset.GetRasterBand(1)  # Access the first (and only) band

# Read the pixel data into a NumPy array
data = band.ReadAsArray()

# Create a plot
plt.figure(figsize=(8, 8))  # Set the figure size
plt.imshow(data, cmap='gray')  # Plot the data as a grayscale image

# Add a colorbar for reference
plt.colorbar(label='Pixel Value')

# Set axis labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Single-Band Satellite Image')

# Display the plot
plt.show()

# Close the dataset
dataset = None

In [ ]:
import matplotlib.pyplot as plt
from osgeo import gdal
import numpy as np

# Open the RGB satellite image using GDAL
dataset = gdal.Open("path/to/rgb_image.tif")

# Read individual bands
red_band = dataset.GetRasterBand(1)
green_band = dataset.GetRasterBand(2)
blue_band = dataset.GetRasterBand(3)

# Read the pixel data into NumPy arrays
red_data = red_band.ReadAsArray()
green_data = green_band.ReadAsArray()
blue_data = blue_band.ReadAsArray()

# Stack the RGB bands to create a true-color image
rgb_image = np.stack((red_data, green_data, blue_data), axis=-1)

# Create a plot
plt.figure(figsize=(8, 8))  # Set the figure size
plt.imshow(rgb_image)  # Plot the RGB image

# Set axis labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('RGB Satellite Image')

# Display the plot
plt.show()

# Close the dataset
dataset = None

## <span style="color:black;">Crop image</span>

```sh
gdalinfo LC08_L2SP_022039_20190322_20200829_02_T1_SR_B2.TIF
```

In [ ]:
from osgeo import gdal, ogr, gdal_translate
input_image_path = "path/to/input_image.tif"
input_dataset = gdal.Open(input_image_path)

shapefile_path = "path/to/shapefile.shp"
shapefile_ds = ogr.Open(shapefile_path)
layer = shapefile_ds.GetLayer()

feature = layer.GetNextFeature()
cropping_geometry = feature.GetGeometryRef()

output_image_path = "path/to/output_image.tif"
gdal.Warp(output_image_path, input_dataset, cutlineDSName=shapefile_path, cutlineLayer=layer.GetName(), dstNodata=0)

input_dataset = None
shapefile_ds = None


In [ ]:
def write_geotiff(fname, data, geo_transform, projection):
    """Create a GeoTIFF file with the given data."""
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None  # Close the file

In [ ]:
# MOSAIC

import glob
from osgeo import gdal

# Get a list of all the image files
image_files = glob.glob('*.tif')

# Create a VRT file
vrt_file = 'mosaic.vrt'
vrt_options = gdal.BuildVRTOptions(resampleAlg='average')
gdal.BuildVRT(vrt_file, image_files, options=vrt_options)

# Read the VRT file and save it as a new mosaic image
mosaic_ds = gdal.Open(vrt_file)
mosaic_data = mosaic_ds.ReadAsArray()
mosaic_trans = mosaic_ds.GetGeoTransform()
mosaic_proj = mosaic_ds.GetProjection()

mosaic_file = 'mosaic.tif'
mosaic_driver = gdal.GetDriverByName('GTiff')
mosaic_ds = mosaic_driver.Create(mosaic_file, mosaic_data.shape[1], mosaic_data.shape[0], 1, gdal.GDT_Float32)
mosaic_ds.SetGeoTransform(mosaic_trans)
mosaic_ds.SetProjection(mosaic_proj)
mosaic_ds.GetRasterBand(1).WriteArray(mosaic_data)

# Close the datasets
mosaic_ds = None
mosaic_ds = None

In [ ]:
def geo_to_pixel_coords(x, y, geotrans, cols, rows):
    # type: (float, float, tuple, int, int) -> (int, int)
    """
    Convert geo-coordinates to grid-coordinates

    :param x: x-coordinate
    :param y: y-coordinate
    :param cols: number of columns of input image (x-size)
    :param rows: number of rows of input image (y-size)
    :param geotrans: GeoTransformObject from gdalDataSource or tuple with (xMin, cellWidth,
            xRotation, yMax, yRotation, cellHeight). cellHeight has to be negative!
    :return: Column and row at given position (col, row)
    """
    ulx = geotrans[0]
    uly = geotrans[3]
    lrx = geotrans[0] + cols * geotrans[1]
    lry = geotrans[3] + rows * geotrans[5]
    if x < ulx or x > lrx or y < lry or y > uly:
        raise ValueError('Coordinate is outside the image!')
    # do the math
    x_pix = int(math.floor((x - ulx) / geotrans[1]))
    y_pix = int(math.floor((uly - y) / -geotrans[5]))
    return x_pix, y_pix


def pixel_to_geo_coords(col, row, geotrans, loc='center'):
    # type: (int, int, tuple, str) -> (float, float)
    """
    Convert grid-coordinates to geo-coordinates

    :param col: column number (counting starts at 0)
    :param row: row number (counting starts at 0)
    :param geotrans: GeoTransformObject from gdalDataSource or tuple with (xMin, cellWidth,
            xRotation, yMax, yRotation, cellHeight). cellHeight has to be nagtive!
    :param loc: location within the pixel. One of "center", "ul", "ur", "ll", "lr"
    :return: Coordinates at given grid cell (x, y)
    """
    ulx = geotrans[0]
    xres = geotrans[1]
    xrot = geotrans[2]
    uly = geotrans[3]
    yrot = geotrans[4]
    yres = geotrans[5]
    # do the math
    if loc == 'center':
        xgeo = ulx + col * xres + xres / 2. + row * xrot
        ygeo = uly + row * yres + yres / 2. + col * yrot
    elif loc == 'ul':
        xgeo = ulx + col * xres + row * xrot
        ygeo = uly + row * yres + col * yrot
    elif loc == 'll':
        xgeo = ulx + col * xres + row * xrot
        ygeo = uly + row * yres + yres + col * yrot
    elif loc == 'ur':
        xgeo = ulx + col * xres + xres + row * xrot
        ygeo = uly + row * yres + col * yrot
    elif loc == 'lr':
        xgeo = ulx + col * xres + xres + row * xrot
        ygeo = uly + row * yres + yres + col * yrot
    else:
        raise ValueError('loc only takes one of "center", "ul", "ur", "ll", "lr"!')
    return xgeo, ygeo


In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt

ds = gdal.Open("dem.tif")

# reproject
dsReprj = gdal.Warp("demReprj.tif", ds, dstSRS = "EPSG:4326")
# resample
dsRes = gdal.Warp("demRes.tif", ds, xRes = 150, yRes = 150,
                  resampleAlg = "bilinear")
# extraction
# make sure your rastermgt data and shapefile have the same projection!
dsClip = gdal.Warp("demClip.tif", ds, cutlineDSName = "star.shp",
                   cropToCutline = True, dstNodata = np.nan)

# visualize
array = dsClip.GetRasterBand(1).ReadAsArray()
plt.figure()
plt.imshow(array)
plt.colorbar()
plt.show()

# close your datasets!
ds = dsClip = dsRes = dsReprj = None

In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt

srcfile = r'V:\temp\sentinel.tif'
dstfile = r'V:\temp\sentinel_out.tif'

# reproject
ds = gdal.Open(srcfile)
dsReprj = gdal.Warp(dstfile, ds, dstSRS="EPSG:4326")

# resample
srcfile = r'V:\temp\sentinel.tif'
dstfile = r'V:\temp\sentinel_out2.tif'
ds = gdal.Open(srcfile)
dsRes = gdal.Warp(dstfile, ds, xRes=100, yRes=100, resampleAlg="bilinear")

# extraction
srcfile = r'V:\temp\sentinel.tif'
src_shp = r'V:\temp\tt.shp'
dstfile = r'V:\temp\sentinel_out4.tif'

ds = gdal.Open(srcfile)
# make sure your rastermgt data and shapefile have the same projection!
dsClip = gdal.Warp(dstfile, ds, cutlineDSName=src_shp, cropToCutline=True, dstNodata=0)

# close your datasets!
ds = dsClip = dsRes = dsReprj = None

In [ ]:
from osgeo import gdal

def export_image_2d(path_output_img, d2array, ds_gdal):

    # create image driver
    driver = gdal.GetDriverByName('GTiff')
    # create destination for label file
    file = driver.Create(path_output_img,
                         d2array.shape[1],
                         d2array.shape[0],
                         1,
                         gdal.GDT_Byte, ['COMPRESS=LZW'])
    file.SetGeoTransform(ds_gdal.GetGeoTransform())
    file.SetProjection(ds_gdal.GetProjection())
    file.GetRasterBand(1).SetNoDataValue(0)
    # write label file
    file.GetRasterBand(1).WriteArray(d2array)
    file.FlushCache()  ##saves to disk!!
    file = None
    ds = None
    driver = None

## <span style="color:black;">Gdal_Translate</span>

To run, replace the input for the landsat image (LC08_L2SP_022039_20190322_20200829_02_T1_SR_B2.tif)

List raster drivers
```sh
> gdal_translate --formats
```

Convert between raster formats
```sh
> gdal_translate -of "JP2OpenJPEG" input.tif output.jp2
```

```sh
> gdal_translate -of "ENVI" input.tif output.envi
```

Compress my data in LZW
```sh
> gdal_translate -of "GTiff" -co "COMPRESS=LZW" LC08_L2SP_022039_20190322_20200829_02_T1_SR_B2.TIF output_compress.tif
```


## <span style="color:black;">GdalWarp</span>

The gdalwarp has many applications, including reprojection, mosaicing, resampling pixel size, set nodata, cut the raster with other shapefile, and more.
To understand the full list of options, see the parameters in [gdalwarp page](https://gdal.org/programs/gdalwarp.html#gdalwarp)


Clip raster with shapefile for pixels within polygon boundary
```sh
> gdalwarp -crop_to_cutline -cutline lake_within_landsat.shp input.tif output.tif
```

Change no data
```sh
> gdalwarp -srcnodata -999999 -dstnodata 0 input.tif output.tif
```

Reproject raster
```sh
> gdalwarp -srcnodata -t_srs "EPSG:4326" input.tif output.tif
```

## <span style="color:black;">Gdal_Build_VRT</span>

Merge rasters

```sh
> dir /b *.tif > filelist.txt
> gdalbuildvrt -input_file_list filelist.txt merged.vrt
> gdalwarp merged.vrt output.tif
```